In [6]:
import sys
project_root_path = '../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
import commit_stats

In [7]:
import json
in_path = project_root_path + '/python/cfg/ds_pipeline.json'
config_map = json.loads(open(in_path, 'r').read())
config_map

cfg = config_map['matt_language_analyzer'] # 99_problems

In [8]:

path = project_root_path + '/' + cfg['stats_dir']
all_commit_stats = commit_stats.load_commit_stats(cfg['stat_file_name'], path)
all_commit_stats[slice(0,2)]

[{'commit': 'f397110afc44e27afc9eeaaf9b5ced3e3d5cc231',
  'Author': 'Navin Moorthy <navin007.a@gmail.com>',
  'Date': '2022-06-29T00:31:28-07:00',
  'orig_timezone': 'UTC+05:30',
  'fileTypes': {'js': {'inserts': 0, 'deletes': 4, 'occurrences': 1},
   'json': {'inserts': 1, 'deletes': 2, 'occurrences': 1},
   'lock': {'inserts': 254, 'deletes': 122, 'occurrences': 1}},
  'file_list': {'.storybook/preview.js': {'binary': 0,
    'inserts': 0,
    'deletes': 4},
   'package.json': {'binary': 0, 'inserts': 1, 'deletes': 2},
   'yarn.lock': {'binary': 0, 'inserts': 254, 'deletes': 122}},
  'seen_in': ['../active'],
  'author_hash': '5fe5c185c7ecd8af13b2c6049ab7a6ac',
  'iso_datetime': '2022-06-29T00:31:28-07:00',
  'utc_datetime': '2022-06-29T07:31:28'},
 {'commit': '151a07655991f3aeff23c1d4096d3eaa4753d71e',
  'Author': 'mcnaveen <8493007+mcnaveen@users.noreply.github.com>',
  'Date': '2022-06-29T00:24:38-07:00',
  'orig_timezone': 'UTC+05:30',
  'fileTypes': {'tsx': {'inserts': 6, 'delete

In [9]:
ft_dicts = []
ft_dict_dict = {}
for stats in all_commit_stats:
    for file_type in stats['fileTypes']:
        commit_ft_dict = stats['fileTypes'][file_type]
        if file_type not in ft_dict_dict:
            ft_dict = {}
            ft_dicts.append(ft_dict)
            ft_dict_dict[file_type] = ft_dict
            ft_dict['type'] = file_type
            ft_dict['lines'] = 0
            ft_dict['files'] = 0
            ft_dict['commits'] = 0
            ft_dict['repos'] = {}
        ft_dict = ft_dict_dict[file_type]
        ft_dict['commits'] += 1
        ft_dict['lines'] += commit_ft_dict['inserts']
        ft_dict['files'] += commit_ft_dict['occurrences']
        #seen_in = commit_ft_dict['seen_in']
        #for project in seen_in:
        #    ft_dict['repos'][project] = 1


In [10]:
by_commit_count = sorted(ft_dicts, key=lambda d: d['commits'])
by_commit_count.reverse()
# if your odds of being in a commit are < 1/1000, you don't matter
main_type_stats = [stats for stats in by_commit_count if stats['commits'] > (len(all_commit_stats) / 1000)]
print(len(main_type_stats))
main_type_stats

65


[{'type': 'js',
  'lines': 2769798,
  'files': 22882,
  'commits': 3806,
  'repos': {}},
 {'type': 'cpp',
  'lines': 503682,
  'files': 14745,
  'commits': 3764,
  'repos': {}},
 {'type': 'noexttext',
  'lines': 41641,
  'files': 5299,
  'commits': 3681,
  'repos': {}},
 {'type': 'md', 'lines': 191608, 'files': 6200, 'commits': 3267, 'repos': {}},
 {'type': 'php',
  'lines': 775800,
  'files': 17278,
  'commits': 3261,
  'repos': {}},
 {'type': 'h', 'lines': 503516, 'files': 8951, 'commits': 2785, 'repos': {}},
 {'type': 'c', 'lines': 378934, 'files': 5470, 'commits': 2688, 'repos': {}},
 {'type': 'py', 'lines': 347350, 'files': 6117, 'commits': 2447, 'repos': {}},
 {'type': 'txt', 'lines': 233308, 'files': 7220, 'commits': 1919, 'repos': {}},
 {'type': 'gdoc',
  'lines': 289925,
  'files': 14561,
  'commits': 1616,
  'repos': {}},
 {'type': 'ts', 'lines': 170667, 'files': 8357, 'commits': 1612, 'repos': {}},
 {'type': 'json',
  'lines': 579827,
  'files': 4275,
  'commits': 1394,
  'r

In [11]:
import pandas as pd

main_types = [stat['type'] for stat in main_type_stats]
main_types.index('json')
rows = []
for commit_stats in all_commit_stats:
    row = [0]*len(main_types)
    rows.append(row)
    for file_type in commit_stats['fileTypes']:
        commit_ft_dict = commit_stats['fileTypes'][file_type]
        if file_type in main_types:
            position = main_types.index(file_type)
            row[position] = commit_ft_dict['inserts']
type_df = pd.DataFrame(rows, columns=main_types) # index=commit_ids,
type_df

,js,cpp,noexttext,md,php,h,c,py,txt,gdoc,...,scss,toml,plist,vcxproj,common,vcproj,cmake_in,conf,tscn,Rmd
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31290,0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31291,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31292,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31293,0,0,0,0,9,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
type_corr = type_df.corr()
print(str(type_corr))

                 js       cpp  noexttext        md       php         h  \
js         1.000000 -0.000600   0.716697  0.985517  0.000112 -0.000480   
cpp       -0.000600  1.000000   0.004428 -0.000217 -0.000491  0.508307   
noexttext  0.716697  0.004428   1.000000  0.713219  0.016528  0.005950   
md         0.985517 -0.000217   0.713219  1.000000 -0.000238  0.001778   
php        0.000112 -0.000491   0.016528 -0.000238  1.000000 -0.000483   
...             ...       ...        ...       ...       ...       ...   
vcproj    -0.000181  0.013439   0.000609 -0.000240 -0.000148  0.003814   
cmake_in  -0.000092 -0.000244  -0.000281 -0.000121 -0.000075 -0.000218   
conf       0.005596  0.003934   0.006943  0.005785 -0.000091 -0.000292   
tscn      -0.000149 -0.000397   0.000758  0.003237 -0.000122 -0.000390   
Rmd        0.000581 -0.000523  -0.000268 -0.000163 -0.000180 -0.000576   

                  c        py       txt      gdoc  ...      scss      toml  \
js        -0.000612  0.958320  0.

In [13]:
for i in range(len(main_types)):
    for j in range(len(main_types)):
        if i < j:
            val = type_corr.iat[i, j]
            if val > 0.2:
                print(main_types[i] + ", " + main_types[j] + ": " + str(val))

js, noexttext: 0.7166967463468783
js, md: 0.985516999667076
js, py: 0.9583197840907771
js, json: 0.23691106535167572
js, rst: 0.9854641393752397
js, html: 0.9051583956310943
js, css: 0.9961492114503521
js, sh: 0.9058727002927767
js, png: 0.9960087710642974
js, svg: 0.4800205840443674
js, scss: 0.9960261766372768
cpp, h: 0.5083074651699927
cpp, hpp: 0.3172439433628845
cpp, lua: 0.298729015284515
noexttext, md: 0.7132188088567121
noexttext, py: 0.6913572100603381
noexttext, rst: 0.7114341360337113
noexttext, html: 0.653882912200029
noexttext, css: 0.7185540251938758
noexttext, sh: 0.6780248120860033
noexttext, png: 0.7185167540654768
noexttext, svg: 0.3408580296635438
noexttext, scss: 0.7180967771478263
md, py: 0.9524054844480578
md, ts: 0.20359893771530307
md, json: 0.23532271327778206
md, rst: 0.9787428040104424
md, html: 0.8988479823088071
md, css: 0.9881721633462345
md, sh: 0.9083595636894067
md, png: 0.9892436951811204
md, svg: 0.463793436620774
md, scss: 0.9884302278299489
h, c: 0.